In [139]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import nltk
from nltk import sent_tokenize
import string
from sklearn.cluster import KMeans
import time
import pandas as pd
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration
import random
from itertools import combinations


tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

device = torch.device('cuda:0' if torch.cuda.is_available() else 0)
model.to(device)

def make_summarization(sentences, tokenizer, model): # 문자열 생성 요약
    if(len(sentences) < 4): return [max(sentences, key=lambda x:len(x))]
    split = []
    for i in range(len(sentences)//8):
        split.append(sentences[:8])
        sentences = sentences[8:]

    for i in range(len(split)):
        if(len(sentences) == 0): break
        split[i].append(sentences.pop())
    
    if(len(sentences) != 0): split.append(sentences)
    
    split_sum = []
    for sentences in split:
        text = '\n'.join(sentences)
        start = time.time()
        raw_input_ids = tokenizer.encode(text)
        input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
        
        #model.generate(torch.tensor([input_ids[:1020]]).to(device), num_beams=3, max_length=256, eos_token_id=1)
        summary_ids = model.generate(torch.tensor([input_ids]).to(device),  num_beams=4,  max_length=256, min_length=10,  eos_token_id=1)
        print(f"{time.time()-start:.4f} sec")
        sum_result = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
        sum_result = sent_tokenize(sum_result)[0]
        sum_result = delete_repeat_str(sum_result)
        print(sum_result)
        split_sum.append(sum_result)
        print(len(split), len(split_sum))
        print('-----------------------------------------------')
    
    if(len(split_sum)==1):
        return [split_sum[0]]
      
    return split_sum


def summarize_topic(document_df, topic_num, tokenizer, model): # df와 topic num을 넣으면 해당 topic num을 요약
    sentences = []
    numbers = []
    for i,t in enumerate(document_df[document_df['cluster_label'] == topic_num]['opinion_text']):
        sentences.append(eval(t)[1])
        numbers.append(eval(t)[0])
    result = make_summarization(sentences, tokenizer, model)
    avg = sum(numbers)/len(numbers)
    return (avg, result)


def summarize_first_sentences(processed_sentences, tokenizer, model): # 문쟈열을 k-means로 토픽 별 분류(첫 문장)
    clusternum = 1
    document_df = get_clustered_df(processed_sentences, clusternum)
    sum_result = []
    for c in range(clusternum):
        temp = get_topic_sentences(document_df, c)
        summ = summarize_topic(document_df, c, tokenizer, model)
        sum_result.append(summ)
        print('===================================================')
    
    first_result = (sum_result[0][0], [min(sum_result[0][1], key=lambda x:len(x))])
    print(first_result)
    return [first_result]
    

def summarize_topk_sentences(processed_sentences, tokenizer, model): # 문쟈열을 k-means로 토픽 별 분류
    clusternum = max(len(processed_sentences)//7, 1)
    document_df = get_clustered_df(processed_sentences, clusternum)
    sum_result = []
    for c in range(clusternum):
        temp = get_topic_sentences(document_df, c)
        print('---------------------------------------------------')
        summ = summarize_topic(document_df, c, tokenizer, model)
        sum_result.append(summ)
        print(summ)
        print('***************************************************')
        
    return sorted(sum_result, key= lambda x: x[0])


def get_clustered_df(sentences, clusternum):
    print(clusternum)
    
    document_df = pd.DataFrame()
    document_df['opinion_text'] = [str(t) for t in sentences]
    
    if(len(sentences) < 2):
        document_df['cluster_label'] = 0
        print('len document df', len(document_df))
        return document_df.sort_values(by=['cluster_label'])
    
    remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

    lemmar = WordNetLemmatizer()
    
    # 토큰화한 각 단어들의 원형들을 리스트로 담아서 반환
    def LemTokens(tokens):
        return [lemmar.lemmatize(token) for token in tokens]

    # 텍스트를 Input으로 넣어서 토큰화시키고 토큰화된 단어들의 원형들을 리스트로 담아 반환
    def LemNormalize(text):
        return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))
    
    tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize,
                            ngram_range=(1,2),
                            min_df=0.05, max_df=0.85)


    ftr_vect = tfidf_vect.fit_transform(document_df['opinion_text'])
    kmeans = KMeans(n_clusters=clusternum, max_iter=10000, random_state=42)
    cluster_label = kmeans.fit_predict(ftr_vect)
    
    # 군집화한 레이블값들을 document_df 에 추가하기
    document_df['cluster_label'] = cluster_label
    return document_df.sort_values(by=['cluster_label'])
    

def get_topic_sentences(df, clabel):
    lst = []
    for i,t in enumerate(df[df['cluster_label'] == clabel]['opinion_text']):
        print(i, t)
        lst.append(t)
    return lst 


def delete_similar(input_sentences):
    if(len(input_sentences) < 2):
        return input_sentences
    sorted_sentences = sorted(input_sentences, key=lambda x:x[1][::-1])
    prev_len = len(sorted_sentences)
    
    for i in range(5):
        prev = sorted_sentences[0]
        processed_sentences = []
        for j,sentence in enumerate(sorted_sentences[1:]):
            s1 = set(prev[1].split())
            s2 = set(sentence[1].split())
            actual_jaccard = float(len(s1.intersection(s2)))/float(len(s1.union(s2)))
            if(actual_jaccard < 0.5): # if not similar
                processed_sentences.append(prev)
                prev = sentence
            else:
                print(prev)
                print(sentence)
                print(actual_jaccard)
                print('-------------------------------------------')
                
        s1 = set(prev[1].split())
        
        if(len(processed_sentences) == 0):
            processed_sentences.append(prev)
            return processed_sentences
        
        s2 = set(processed_sentences[-1][1].split())
        actual_jaccard = float(len(s1.intersection(s2)))/float(len(s1.union(s2)))
        if(actual_jaccard < 0.5): # if not similar
            processed_sentences.append(prev)
        
        sorted_sentences = sorted(processed_sentences, key=lambda x:x[1])
        
        if(prev_len == len(sorted_sentences)):
            break
        prev_len = len(sorted_sentences)
        
    return sorted_sentences


def get_first_topk_sentences(df, topk_percent):
    first_sentences = []
    topk_sentences = []
    for a,b in zip(df['context2'], df['topk']):
        context = eval(str(a))
        topk = eval(str(b))
        k = int(len(topk)*(topk_percent/100))
        topk = topk[:k]
        
        first = []
        for item in topk:
            if(item[0] == 0): 
                first.append(item)
                
        if(len(first) == 0):
            first_sentences += [(0, context[0])]
            topk_sentences += topk
        else:
            first_sentences += first
            topk.remove(first[0])
            topk_sentences += topk
                
    print('before delete similar:', len(first_sentences), len(topk_sentences))
    first_sentences = delete_similar(first_sentences)
    topk_sentences = delete_similar(topk_sentences)
    print('after delete similar:', len(first_sentences), len(topk_sentences))
    return first_sentences, topk_sentences


def get_additional_topk_sentences(df, prev_topk_percent, next_topk_percent):
    topk_sentences = []
    next_topk_percent = prev_topk_percent + next_topk_percent
    for a,b in zip(df['context2'], df['topk']):
        context = eval(str(a))
        topk = eval(str(b))
        pk = int(len(topk)*(prev_topk_percent/100))
        k = int(len(topk)*(next_topk_percent/100))
        topk = topk[pk:k]
        topk_sentences += topk
                
    print('before delete similar:', len(topk_sentences))
    if(len(topk_sentences) == 0):
        return topk_sentences
    
    topk_sentences = delete_similar(topk_sentences)
    print('after delete similar:', len(topk_sentences))
    return topk_sentences    


def get_topk_sentences(k, user_input, model, tokenizer):
    bot_input_ids = News_to_input(user_input, openapi_key)
    
    chat_history_ids = summary(args, bot_input_ids, -1, '', None, model)
    pred_lst = list(chat_history_ids[0][:k])
    final_text = []
    for i,a in enumerate(user_input.split('.')):
        if i in pred_lst:
            final_text.append(a+'. ')
    return final_text
    


def delete_repeat(user_text):
    text = user_text.split()
    x = len(text)
    comb = list(combinations(range(x), 2))
    sorted_comb = sorted(comb, key=lambda x: x[1]-x[0], reverse = True)
    for item in sorted_comb:
        start, end = item
        if(end-start <= len(sorted_comb)/2 and end-start>2):
            find_str = ' '.join(text[start:end])
            rest_str = ' '.join(text[end:])
            idx = rest_str.find(find_str)
            if idx != -1:
                print('deleted :', idx, '|', find_str)
                print('ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ')
                ret = ' '.join(text[:end]) + ' ' + rest_str[:idx] + rest_str[idx+len(find_str)+1:]
                print(user_text)
                print('ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ')
                print(ret)
                print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
                return ret
    return user_text


def delete_repeat_str(user_text):
    prev = user_text
    new = delete_repeat(prev)
    while(new != prev):
        prev = new
        new = delete_repeat_str(prev)        
    return new


def calc_len(sum_result2):
    ret = 0
    for s in sum_result2:
        tmp = 0
        for c in s[1]:
            tmp += len(c)
        ret += tmp
    return ret

In [140]:
import os
from os import listdir
from os.path import isfile, join, isdir

all_paths = [
 'bertopic(mpnet)',
 'bertopic(paraphrase)',
 'kmeans_12',
 'hdbscan',
 'kmeans_8',
 'bertopic(SR-BERT)',
 'kmeans_4']

i = 0
mypath = f'./preprocessed2/{all_paths[i]}'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles

['bertopic_부동산_20221201_20221203_pre2.csv',
 'bertopic_삼성전자_20221201_20221231_pre2.csv',
 'bertopic_부동산_20221201_20221231_pre2.csv',
 'bertopic_부동산_20221201_20221215_pre2.csv',
 'bertopic_삼성전자_20221201_20221203_pre2.csv',
 'bertopic_윤석열_20221201_20221203_pre2.csv',
 'bertopic_윤석열_20221201_20221215_pre2.csv',
 'bertopic_삼성전자_20221201_20221215_pre2.csv',
 'bertopic_윤석열_20221201_20221231_pre2.csv']

In [141]:
j = 2
all_df = pd.read_csv(f'{mypath}/{onlyfiles[j]}')
print(len(all_df))
all_df.topic.unique()

278


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55])

In [146]:
t = 39
print('Topic:', t)
topic_context_df = all_df[all_df.topic == t]
len(topic_context_df)

Topic: 39


12

In [150]:
topic_context_df.raw[185]

'■ 진행 : 안보라 앵커\n\n■ 출연 : 윤지해 부동산R114 수석연구원\n\n\n\n* 아래 텍스트는 실제 방송 내용과 차이가 있을 수 있으니 보다 정확한 내용은 방송으로 확인하시기 바랍니다. 인용 시 [YTN 뉴스라이더] 명시해주시기 바랍니다.\n\n\n\n[앵커]\n\n추운 날씨만큼이나 부동산 시장에도 역대급 한파가 찾아왔습니다. 98년 외환위기 이후, 가장 극심한 침체라는 분석도 있습니다. 윤지해 부동산R114 수석연구원과 함께 전망해 봅니다. 어서 오세요.\n\n\n\n[윤지해]\n\n안녕하세요.\n\n\n\n[앵커]\n\n추운데 잘 오셨습니까?\n\n\n\n[윤지해]\n\n많이 춥네요.\n\n\n\n[앵커]\n\n오늘 날씨만큼이나 시장도 정말 춥네요. 그런데 요즘 깡통전세, 전세사기 이런 뉴스 저도 많이 전하거든요. 결국 집값이 떨어진 게 큰 이유라고 봐야 됩니까?\n\n\n\n[윤지해]\n\n지난 1년을 돌아보시면 우리가 급격히 상승하는 것들을 봤잖아요. 거기서도 왜곡이 굉장히 많이 일어나고요. 지금처럼 급격하게 하락을 하게 되면 시장에서도 상당히 이렇게 역전세 같은 그런 왜곡들이 일어납니다.\n\n\n\n[앵커]\n\n저희가 부동산 시장 얼었다, 얼었다, 이렇게 표현했는데 수치를 보니까 1월부터 11월까지 전국 아파트 값이 4.79%가 하락했습니다. 이게 한국부동산원이 조사한 건데 시세 조사한 이후로 가장 큰 하락폭이라고 합니다. 가장 큰 원인이라고 보세요?\n\n\n\n[윤지해]\n\n원인은 크게 세 가지입니다. 크게 봐서는 세 가지인데 일단 첫 번째는 이미 아시는 것처럼 급격하게 금리가 인상이 된 부분 하나가 있었고요. 그리고 추가적으로 정부가 지난해 10월에 대출규제, 그러니까 DSR이라고 하는 총부채원리금상환규제를 도입하다 보니까 아무래도 유동성이 축소되는 이슈가 있었고요. 거기에 최근 한 5년 사이에 많이 오르지 않았습니까? 그러다 보니까 수요자들이 가격에 대한 저항감이 커졌다, 이렇게 세 가지 이유로 보시면 될 것 같습니다.\n\n

In [147]:
start = time.time()
first_sentences, topk_sentences = get_first_topk_sentences(topic_context_df, 15)

sum_result1 = summarize_first_sentences(first_sentences, tokenizer, model)
sum_result2 = sum_result1 + summarize_topk_sentences(topk_sentences, tokenizer, model)

prev_topk = 15

threshold = min(570, len(topic_context_df) * 100)
while(calc_len(sum_result2) < threshold): # 너무 짧을 때
    print(f'.................{calc_len(sum_result2)}<{threshold} get additional topk {prev_topk} ...............')
    topk_sentences2 = get_additional_topk_sentences(topic_context_df, prev_topk, 7)
    if(len(topk_sentences2) > 0):
        sum_result3 = summarize_topk_sentences(topk_sentences2, tokenizer, model)
        sum_result2 = sorted(sum_result3 + sum_result2, key=lambda x:x[0])
    prev_topk += 7

final_result = ''
for v in sum_result2:
    paragraph = [s.strip() for s in v[1]]
    new_paragraph = [s if('.' in s[-2: ]) else s+'. ' for s in paragraph]
    final_result += ' '.join(new_paragraph)+'\n\n'


print('======================= Final result =========================')
print(final_result)
print(f"{time.time()-start:.4f} sec")
print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
f = open(f'./bertopic(mpnet)/{name}_summary_{i}.txt', 'w')
f.write(final_result)
f.close()

before delete similar: 12 26
(0, '* 아래 텍스트는 실제 방송 내용과 차이가 있을 수 있으니 보다 정확한 내용은 방송으로 확인하시기 바랍니다. ')
(0, '* 아래 텍스트는 실제 방송 내용과 차이가 있을 수 있으니 보다 정확한 내용은 방송으로 확인하시기 바랍니다. ')
1.0
-------------------------------------------
(0, '* 아래 텍스트는 실제 방송 내용과 차이가 있을 수 있으니 보다 정확한 내용은 방송으로 확인하시기 바랍니다. ')
(0, '* 아래 텍스트는 실제 방송 내용과 차이가 있을 수 있으니 보다 정확한 내용은 방송으로 확인하시기 바랍니다. ')
1.0
-------------------------------------------
(0, '* 아래 텍스트는 실제 방송 내용과 차이가 있을 수 있으니 보다 정확한 내용은 방송으로 확인하시기 바랍니다. ')
(0, '* 아래 텍스트는 실제 방송 내용과 차이가 있을 수 있으니 보다 정확한 내용은 방송으로 확인하시기 바랍니다. ')
1.0
-------------------------------------------
(0, '* 아래 텍스트는 실제 방송 내용과 차이가 있을 수 있으니 보다 정확한 내용은 방송으로 확인하시기 바랍니다. ')
(0, '* 아래 텍스트는 실제 방송 내용과 차이가 있을 수 있으니 보다 정확한 내용은 방송으로 확인하시기 바랍니다. ')
1.0
-------------------------------------------
(0, '* 아래 텍스트는 실제 방송 내용과 차이가 있을 수 있으니 보다 정확한 내용은 방송으로 확인하시기 바랍니다. ')
(0, '* 아래 텍스트는 실제 방송 내용과 차이가 있을 수 있으니 보다 정확한 내용은 방송으로 확인하시기 바랍니다. ')
1.0
-------------------------------------------
(0, '* 아래 텍스트는 실제 방송 내용과 차이가 있을 수 있으니 보다 

/opt/conda/envs/python3.6/lib/python3.6/site-packages/transformers/generation_utils.py:1839: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


0.2407 sec
HUG(주택도시보증공사)를 통해서 가입하는 경우들이 있었던 것 같은데, 대위변제 절차를 밟지 못하고 있다고 한다.
2 1
-----------------------------------------------
0.2292 sec
내년 상반 정도는 시장 동향을 보고 미분양이라든지 급격하게 내려간 급매 정도는 충분히 검토할 타이밍도 다가온다고 보여집니다.
2 2
-----------------------------------------------
(10.4, ['HUG(주택도시보증공사)를 통해서 가입하는 경우들이 있었던 것 같은데, 대위변제 절차를 밟지 못하고 있다고 한다.', '내년 상반 정도는 시장 동향을 보고 미분양이라든지 급격하게 내려간 급매 정도는 충분히 검토할 타이밍도 다가온다고 보여집니다.'])
***************************************************
0 (10, ' 예를 들어서 세종시 같은 경우는 2020년에만 40~50%가 올랐습니다. ')
1 (10, ' 한마디로 시장이 롤러코스터를 타고 있다. ')
2 (10, ' 가장 큰 영향을 미친 건 역시 물가라고 볼 수 있겠습니다. ')
3 (10, ' ◆ 조세영: 그렇습니다. ')
---------------------------------------------------


/opt/conda/envs/python3.6/lib/python3.6/site-packages/transformers/generation_utils.py:1839: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


0.2401 sec
카카오 한마디로 시장이 롤러코스터를 타고 롤러코스터를 타고 있는 가운데 가장 큰 영향을 미친 건 역시 물가라고 볼 수 있겠다.
1 1
-----------------------------------------------
(10.0, ['카카오 한마디로 시장이 롤러코스터를 타고 롤러코스터를 타고 있는 가운데 가장 큰 영향을 미친 건 역시 물가라고 볼 수 있겠다.'])
***************************************************
0 (14, ' 통지해야 할 집주인이 사라진 상황이라고 보시면 되겠습니다. ')
1 (7, ' 그래서 이게 역대급이라는 부분의 기록은 이미 세웠지만, 앞으로도 이 부분과 관련해서는 시장이 굉장히 많은 부담으로 작용을 할 것으로 예상이 됩니다. ')
2 (8, ' 그래서 섣불리 아직 바닥 가격이라든가 이른 반등이나 이런 것들을 예상하기가 쉽지 않은 상황이라고 보실 수 있습니다. ')
3 (14, ' 그 경우에 있어서 84제곱미터의 경우에는 6억 원 이상 하락한 것으로 나타나고 있습니다. ')
4 (14, ' 이 얘기는 다시 얘기하자면 앞으로는 경기침체가 정말 본격화될 수 있다는 뜻입니다. ')
5 (15, ' 불분명한 상황이라고 보시면 되겠습니다. ')
6 (5, ' 기준금리의 첫 번째 목표는 물가고 두 번째로는 물가 안정 같은 것들을 이야기하지만 세상에 모든 경제현상에서 기준금리에서 자유로운 경제현상은 없다고 보시면 되겠습니다. ')
---------------------------------------------------


...

0.2194 sec
통지해야 할 집주인이 사라진 상황이라고 보시면 되겠다.
1 1
-----------------------------------------------
(11.0, ['통지해야 할 집주인이 사라진 상황이라고 보시면 되겠다.'])
***************************************************
.................293<570 get additional topk 15 ...............
before delete similar: 13
after delete similar: 13
1
0 (8, ' ◆ 조세영: 네, 그렇습니다. ')
1 (13, ' 개인적인 의견하고 개별적인 여론조사를 통한다면 내년 상반기에 적극적으로 내 집 장만이라든지, 투자 수요가 몰리기에는 좀 한계가 있다고 보여집니다. ')
2 (8, ' 그 경우에는 매매 가격과 전세가가 거의 동일하거나 비슷한 상황입니다. ')
3 (12, ' 그래서 이런 상황들을 보자면 내년 상반기 정도 하락 수준을 판단을 하려면 급매물 중심으로 거래량이 살아나면서 두 자릿수 정도의 하락까지는 좀 더 추가적으로 진행이 돼야 수요자들이 오른 수준에 비해서 적정하게 가격이 떨어지고 있다라고 체감을 하실 수 있지 않을까 싶습니다. ')
4 (6, ' 그런데 그 부분들이 여러 가지 대출이나 시장에 참여하는 많은 분들의 마음마저 얼어붙게 만드는, 그런 것으로 인해서 서울이나 지방 가릴 것 없이 전국의 아파트값이 2003년에 한국부동산원이 집값 조사를 한 이후로 최대의 하락폭을 기록을 한 것 같습니다. ')
5 (24, ' 그런데 여기에서 내년 말 최종 금리 수준을 5에서 5.25%로 제시했습니다. ')
6 (6, ' 당연히 자산시장에도 큰 영향을 미치고 부동산 시장에도 많은 영향을 미칩니다. ')
7 (8, ' 무시무시한 속도로 기준금리를 높였습니다. ')
8 (8, ' 상속을 다 거부하게 되면 그러면 재산관리인을 선임해서 법원에서 진행을 하게 됩니다. ')
9 (10

/opt/conda/envs/python3.6/lib/python3.6/site-packages/transformers/generation_utils.py:1839: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


0.3942 sec
조세영: 내년 상반기 정도 하락 수준을 판단 하려면 급매물 중심으로 거래량이 살아나면서 두 자릿수 정도의 하락까지는 좀 더 추가적으로 진행이 돼야 수요자들이 오른 수준에 비해서 적정하게 가격이 떨어지고 있다라고 체감을 하실 수 있지 않을까 싶다.
2 1
-----------------------------------------------
0.3938 sec
deleted : 5 | 지은이 두 번째 이유는 미국을 이야기하지 않을 수가
ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
지은 우선 우선 네 가지 특성이 큰 원인이었다고 볼 수 있는데,  지은이 재산관리인을 선임해서 법원에서 진행을 하게 되는데  지은이 두 번째 이유는 미국을 이야기하지 않을 수가 없으며,  지은이 두 번째 이유는 미국을 이야기하지 않을 수가 없습니다.
ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
지은 우선 우선 네 가지 특성이 큰 원인이었다고 볼 수 있는데, 지은이 재산관리인을 선임해서 법원에서 진행을 하게 되는데 지은이 두 번째 이유는 미국을 이야기하지 않을 수가 없으며, 없습니다.
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
지은 우선 우선 네 가지 특성이 큰 원인이었다고 볼 수 있는데, 지은이 재산관리인을 선임해서 법원에서 진행을 하게 되는데 지은이 두 번째 이유는 미국을 이야기하지 않을 수가 없으며, 없습니다.
2 2
-----------------------------------------------
(10.307692307692308, ['조세영: 내년 상반기 정도 하락 수준을 판단 하려면 급매물 중심으로 거래량이 살아나면서 두 자릿수 정도의 하락까지는 좀 더 추가적으로 진행이 돼야 수요자들이 오른 수준에 비해서 적정하게 가격이 떨어지고 있다라고 체감을 하실 수 있지 않을까 싶다.', '지은 우선 우선 네 가지 특성이 큰 원인이었다고 볼 수 있는데, 지은이 재산관리인을 선임해서 법원에서 진행을 하게 되는데 지은이 두 번째 이유는 미국을 이야기하지 않을 수가 

/opt/conda/envs/python3.6/lib/python3.6/site-packages/transformers/generation_utils.py:1839: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


0.1830 sec
19년 만에 가장 큰 폭으로 가격이 하락한 이유는 소비자물가상승률로 소비자물가상승률로 봐야 될 것 같다.
2 1
-----------------------------------------------
1.4687 sec
deleted : 0 | 가락 주공아파트를 재건축한 단지라고 볼 수 있는 가락 주공아파트를 재건축한 단지라고 볼 수 있는 가락 주공아파트를 재건축한 단지라고 볼 수 있는 가락 주공아파트를 재건축한 단지라고 볼 수 있는 가락 주공아파트를 재건축한 단지라고 볼 수 있는
ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
가락 주공아파트를 재건축한 단지라고 볼 수 있는 가락 주공아파트를 재건축한 단지라고 볼 수 있는 가락 주공아파트를 재건축한 단지라고 볼 수 있는 가락 주공아파트를 재건축한 단지라고 볼 수 있는 가락 주공아파트를 재건축한 단지라고 볼 수 있는 가락 주공아파트를 재건축한 단지라고 볼 수 있는 가락 주공아파트를 재건축한 단지라고 볼 수 있는 가락 주공아파트를 재건축한 단지라고 볼 수 있는 가락 주공아파트를 재건축한 단지라고 볼 수 있는 가락 주공아파트를 재건축한 단지라고 볼 수 있는 가락 주공아파트를 재건축한 단지이다.
ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
가락 주공아파트를 재건축한 단지라고 볼 수 있는 가락 주공아파트를 재건축한 단지라고 볼 수 있는 가락 주공아파트를 재건축한 단지라고 볼 수 있는 가락 주공아파트를 재건축한 단지라고 볼 수 있는 가락 주공아파트를 재건축한 단지라고 볼 수 있는 가락 주공아파트를 재건축한 단지이다.
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
deleted : 12 | 가락 주공아파트를 재건축한 단지라고 볼 수 있는 가락 주공아파트를 재건축한 단지라고 볼 수 있는 가락 주공아파트를 재건축한
ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
가락 주공아파트를 재건축한 단지라고 볼 수 있는 가락 주공아파트를 재건축한 단지라고 볼 수 있는 가락 주공아파트를 재건축한 단지라고 볼 수 있는 가락 주공아파트를 재